### Introduction

This Notebook can query for restaurants in our general area and compare them to restauraunts we've already done food runs to.  Use it to help decide where to do a lunch run today.

#### Environment setup

In [1]:
from fake_api import search_restaurants, visited_restaurants
import pandas as pd

### Query for yelp-ish data

In [2]:
results = search_restaurants()
len(results)

5

#### Clean data

Read into Pandas and reformat to make it sensible

In [3]:
df = pd.DataFrame(results)
df.head()

,Pizza place,Greek place,Sushi place,Thai place,Tapa place
price,2,1,4,3,4
rating,3,2,3,1,5
nratings,27,5,55,8,9


In [4]:
df = df.T
df.head()

,price,rating,nratings
Pizza place,2,3,27
Greek place,1,2,5
Sushi place,4,3,55
Thai place,3,1,8
Tapa place,4,5,9


### Pull up food-run history

In [5]:
results = visited_restaurants()
len(results)

3

#### Clean data

In [6]:
df2 = pd.DataFrame([results])
df2.head()

,Pizza place,Sushi place,Tapa place
0,"[2019-10-02, 2019-10-09]",[2019-10-03],[2019-10-10]


In [7]:
df2 = df2.T
df2.columns = ['history']
df2.head()

,history
Pizza place,"[2019-10-02, 2019-10-09]"
Sushi place,[2019-10-03]
Tapa place,[2019-10-10]


In [8]:
df2['last_visited'] = df2.history.apply(lambda lst: max(lst))
df2.head()

,history,last_visited
Pizza place,"[2019-10-02, 2019-10-09]",2019-10-09
Sushi place,[2019-10-03],2019-10-03
Tapa place,[2019-10-10],2019-10-10


### Combine data

In [9]:
df = df.join(df2[['last_visited']])
df.head()

,price,rating,nratings,last_visited
Pizza place,2,3,27,2019-10-09
Greek place,1,2,5,NaN
Sushi place,4,3,55,2019-10-03
Thai place,3,1,8,NaN
Tapa place,4,5,9,2019-10-10


### Results

#### All restaurants

##### By price

In [10]:
df.sort_values('price').head()

,price,rating,nratings,last_visited
Greek place,1,2,5,NaN
Pizza place,2,3,27,2019-10-09
Thai place,3,1,8,NaN
Sushi place,4,3,55,2019-10-03
Tapa place,4,5,9,2019-10-10


##### By rating

In [15]:
df.sort_values('rating', ascending=False).head()

,price,rating,nratings,last_visited
Tapa place,4,5,9,2019-10-10
Pizza place,2,3,27,2019-10-09
Sushi place,4,3,55,2019-10-03
Greek place,1,2,5,NaN
Thai place,3,1,8,NaN


#### Unvisited restaurants

In [12]:
unvisited = df[df.last_visited.isnull()]
unvisited.shape

(2, 4)

##### By price

In [13]:
unvisited.sort_values('price').head()

,price,rating,nratings,last_visited
Greek place,1,2,5,NaN
Thai place,3,1,8,NaN


##### By rating

In [16]:
unvisited.sort_values('rating', ascending=False).head()

,price,rating,nratings,last_visited
Greek place,1,2,5,NaN
Thai place,3,1,8,NaN


### CSV output

In [19]:
from IPython.display import FileLink
fname = 'food_run.csv'
df.sort_values(['last_visited', 'rating', 'price'], ascending=False).to_csv(fname, index=False)

FileLink(fname)

/home/jovyan/notebooks/food_run.csv